In [125]:
# Imports

import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib.pyplot as plt

In [128]:
# Get the excel file and read

class ExcelReader(object):
    
    def __init__(self, path, ecID=1):
        self.path = path
        self.ecID = ecID
        
        # Placeholder dictionary to place the next ones
        self.parameterData = {}
        
        # Resources
        self.resources = {}
        
        # General Info dictionary
        self.generalInfo = {}
        
        # Branch dictionary
        self.puSist = {}
        self.network = {}
        
        self.read_generalInfo()
        self.read_brachData()
        
        return
    
    
    def read_generalInfo(self):
        # General information sheet from excel
        generalInfo = pd.read_excel(self.path, sheet_name='General_Information', header=None)
        self.resources['numEC'] = generalInfo.values[2, 3]
        self.resources['period'] = generalInfo.values[3, 3]
        self.resources['periodDuration'] = generalInfo.values[4, 3]
        self.resources['owners'] = generalInfo.values[5, 3]
        
        # Read max imports and export quantities
        self.generalInfo['P_Max_Imp'] = generalInfo.values[23, 5:]
        self.generalInfo['P_Max_Exp'] = generalInfo.values[24, 5:]
        self.generalInfo['Energy_Buy_Price'] = generalInfo.values[25, 5:]
        self.generalInfo['Energy_SellPrice'] = generalInfo.values[26, 5:]
        self.generalInfo['GEE_CofA'] = generalInfo.values[27, 5:]
        self.generalInfo['GEE_CofB'] = generalInfo.values[28, 5:]
        self.generalInfo['GEE_CofC'] = generalInfo.values[29, 5:]
        
        return
    
    
    def read_brachData(self):
        # Branch data sheet from excel
        sheet_name_branch = 'Network_Info'
        data_branch = pd.read_excel(self.path, sheet_name=sheet_name_branch, header=None)
        
        # Set the values
        self.puSist['basePower'] = data_branch.values[1, 8]
        self.puSist['baseVoltage'] = data_branch.values[2, 8]
        self.puSist['baseCurrent'] = data_branch.values[3, 8]
        self.puSist['baseImpendance'] = data_branch.values[4, 8]
        
        busRef = data_branch.values[2, :3]
        busAux = np.unique(data_branch.values[22:, 1:3])
        
        voltageMax = np.zeros((max(busAux), self.resources['period']))
        voltageMin = np.zeros((max(busAux), self.resources['period']))
        angleMax = np.zeros((max(busAux), self.resources['period']))
        angleMin = np.zeros((max(busAux), self.resources['period']))
        
        for i in busAux:
            voltageMax[i-1, :] = data_branch.values[2, 5]
            voltageMin[i-1, :] = data_branch.values[2, 4]
            angleMax[i-1, :] = data_branch.values[4, 5]
            angleMin[i-1, :] = data_branch.values[4, 4]
        
        voltageMax[busRef[0]-1, :] = busRef[1]
        voltageMin[busRef[0]-1, :] = busRef[1]
        angleMax[busRef[0]-1, :] = busRef[2]
        angleMin[busRef[0]-1, :] = busRef[2]
        
        self.network['voltageMax'] = voltageMax
        self.network['voltageMin'] = voltageMin
        self.network['angleMax'] = angleMax
        self.network['angleMin'] = angleMin
        self.network['busNum'] = np.reshape(busAux, (1, busAux.shape[0]))
        self.network['busRef'] = busRef[0]
        
        # Conversion from data_branch
        numBranch = data_branch.values[:, 0].shape[0] - 22
        ybus = np.zeros((max(busAux), max(busAux)))
        diag = np.zeros((max(busAux), max(busAux)))
        branchID = np.zeros((max(busAux), max(busAux)))
        
        busI = data_branch.values[22:, 1]
        busJ = data_branch.values[22:, 2]
        
        # Admitance set
        for i in np.arange(busI.shape[0]):
            ybus[busI[i]-1, busJ[i]-1] = 1
            ybus[busJ[i]-1, busI[i]-1] = 1
            
            branchID[busI[i]-1, busJ[i]-1] = 1
            
            
        # Diagonal matrix values
        for i in np.arange(ybus.shape[0]):
            ybus[i, i] = 1
            diag[i, i] = 1
            
        branch = np.zeros((max(busAux), max(busAux), 4))
        idBusI = np.matlib.repmat(busI, 1, 4)
        idBusI = np.transpose(idBusI)
        
        idBusJ = np.matlib.repmat(busJ, 1, 4)
        idBusJ = np.transpose(idBusJ)
        
        
        #print(idBusI.shape)
        
        #TODO: continue from line 118
        
        return
        

    
    def readExcel(self):

        # Excel sheets to read from the excel
        sheet_name_branch = 'Network_Info'
        sheet_name_peers = 'Peers_Info_EC{}'.format(self.ecID)
        sheet_name_gen = 'Generator_EC{}'.format(self.ecID)
        sheet_name_load = 'Load_EC{}'.format(self.ecID)
        sheet_name_stor = 'Storage_EC{}'.format(self.ecID)
        sheet_name_cstation = 'CStation_EC{}'.format(self.ecID)
        sheet_name_v2g = 'Vehicle_EC{}'.format(self.ecID)

        data_peers = pd.read_excel(path, sheet_name=sheet_name_peers, header=None)
        data_gen = pd.read_excel(path, sheet_name=sheet_name_gen, header=None)
        data_load = pd.read_excel(path, sheet_name=sheet_name_load, header=None)
        data_storage = pd.read_excel(path, sheet_name=sheet_name_stor, header=None)
        data_cstation = pd.read_excel(path, sheet_name=sheet_name_cstation, header=None)
        data_v2g = pd.read_excel(path, sheet_name=sheet_name_v2g, header=None)

        print(data_v2g)

        return generalInfo

data = ExcelReader(path='src/EC_OnlyMembers_V8/EC_V4.xlsx', ecID=1)

(144, 1)


In [103]:
data.network['busRef']

1